# 07 – Subqueries & Views

Core SQL concepts: subqueries, derived tables, and views.

---

*Part of the [Foundations: Python, R & SQL](../README.md) repository.*


In [1]:
import duckdb

In [2]:
# Create table for subquery examples
duckdb.sql("""
CREATE TABLE sales (
    id INTEGER,
    product TEXT,
    quantity INTEGER,
    price FLOAT
);

INSERT INTO sales VALUES
(1, 'Laptop', 2, 999.99),
(2, 'Tablet', 4, 450.00),
(3, 'Laptop', 1, 999.99),
(4, 'Phone', 5, 699.00),
(5, 'Tablet', 2, 450.00);
""")

In [3]:
duckdb.sql("SELECT * FROM sales")

┌───────┬─────────┬──────────┬────────┐
│  id   │ product │ quantity │ price  │
│ int32 │ varchar │  int32   │ float  │
├───────┼─────────┼──────────┼────────┤
│     1 │ Laptop  │        2 │ 999.99 │
│     2 │ Tablet  │        4 │  450.0 │
│     3 │ Laptop  │        1 │ 999.99 │
│     4 │ Phone   │        5 │  699.0 │
│     5 │ Tablet  │        2 │  450.0 │
└───────┴─────────┴──────────┴────────┘

## 1. Subqueries in SELECT

Use subqueries to compute derived values.


In [4]:
# Total revenue per sale
duckdb.sql("""
SELECT *,
       quantity * price AS total
FROM sales;
""")

┌───────┬─────────┬──────────┬────────┬─────────┐
│  id   │ product │ quantity │ price  │  total  │
│ int32 │ varchar │  int32   │ float  │  float  │
├───────┼─────────┼──────────┼────────┼─────────┤
│     1 │ Laptop  │        2 │ 999.99 │ 1999.98 │
│     2 │ Tablet  │        4 │  450.0 │  1800.0 │
│     3 │ Laptop  │        1 │ 999.99 │  999.99 │
│     4 │ Phone   │        5 │  699.0 │  3495.0 │
│     5 │ Tablet  │        2 │  450.0 │   900.0 │
└───────┴─────────┴──────────┴────────┴─────────┘

## 2. Subqueries in WHERE

Filter results using conditions based on another query.


In [5]:
# Show rows where price is higher than the average
duckdb.sql("""
SELECT *
FROM sales
WHERE price > (SELECT AVG(price) FROM sales);
""")

┌───────┬─────────┬──────────┬────────┐
│  id   │ product │ quantity │ price  │
│ int32 │ varchar │  int32   │ float  │
├───────┼─────────┼──────────┼────────┤
│     1 │ Laptop  │        2 │ 999.99 │
│     3 │ Laptop  │        1 │ 999.99 │
└───────┴─────────┴──────────┴────────┘

## 3. Subqueries in FROM

Use derived tables for intermediate computation.


In [6]:
duckdb.sql("""
SELECT product, AVG(quantity) AS avg_quantity
FROM (
    SELECT * FROM sales WHERE product != 'Phone'
)
GROUP BY product;
""")

┌─────────┬──────────────┐
│ product │ avg_quantity │
│ varchar │    double    │
├─────────┼──────────────┤
│ Laptop  │          1.5 │
│ Tablet  │          3.0 │
└─────────┴──────────────┘

## 4. Views

Views are virtual tables based on SQL queries.


In [7]:
# Create a view for high-value sales
duckdb.sql("""
CREATE VIEW high_sales AS
SELECT *
FROM sales
WHERE quantity * price > 2000;
""")

In [8]:
# Query the view
duckdb.sql("SELECT * FROM high_sales")

┌───────┬─────────┬──────────┬───────┐
│  id   │ product │ quantity │ price │
│ int32 │ varchar │  int32   │ float │
├───────┼─────────┼──────────┼───────┤
│     4 │ Phone   │        5 │ 699.0 │
└───────┴─────────┴──────────┴───────┘

## Summary

- Use subqueries to create intermediate logic inside `SELECT`, `WHERE`, and `FROM`.
- Views help simplify repeated queries.